In [1]:
import joblib
import xarray as xr
import pandas as pd
import anndata
import numpy as np
import seaborn as sns
import anndata
from scipy.sparse import csr_matrix, vstack
import matplotlib.pyplot as plt

## Load Data

### Cell type phylogeny

In [2]:
node_cluster_dict = joblib.load(
    '/home/hanliu/project/mouse_rostral_brain/study/AssignGeneToTree/RTree/Inh.non_singleton_node_dict.lib'
)

cluster_dendro = joblib.load(
    '/home/hanliu/project/mouse_rostral_brain/study/AssignGeneToTree/RTree/Inh.dendrogram.lib'
)
cluster_linkage = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/AssignGeneToTree/RTree/Inh.linkage.csv', index_col=0).values
cluster_linkage_order = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/AssignGeneToTree/RTree/Inh.linkage.orders.txt', 
    index_col=0, header=None
).index

cluster_linkage_order = [i.replace(' ', '_') for i in cluster_linkage_order]
name_map = {i.replace('_', '.').replace('-', '.'):i for i in cluster_linkage_order}
cluster_dendro['ivl'] = [name_map[i] for i in cluster_dendro['ivl']]

### DMR

#### DMG DMR Corr

In [3]:
# dmr_gene_corr = pd.read_msgpack(
#     '/home/hanliu/project/mouse_rostral_brain/study/DMRGeneCorr/TotalGeneDMRCorrLoop.0.3.msg')

#### DMR Rate

In [4]:
dmr_rate = pd.read_hdf(
    '/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRInfo.h5',
    key='Rate').loc[:, cluster_dendro['ivl']].copy()

#### DMR Hits

In [5]:
hypo_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalHits.HypoDMR.h5ad'
)[:, cluster_dendro['ivl']].copy()

### Make pairs

In [6]:
from itertools import combinations

In [7]:
cluster_pairs = list(combinations(cluster_dendro['ivl'], 2))
print(len(cluster_pairs), 'pairwise comparison to test.')

2926 pairwise comparison to test.


## Select DMR Per Pair

In [8]:
dmr_rate_delta = 0.4

In [9]:
pairs = []
records = []

for i, (a, b) in enumerate(cluster_pairs):
    rate_delta = dmr_rate[a] - dmr_rate[b]
    # only one of the cluster is 1, means one hypo, the other not
    hypo_judge = (hypo_hits.obs_vector(a) + hypo_hits.obs_vector(b)) == 1
    rate_delta = rate_delta[hypo_judge].copy()
    # top 100k (or less) most diff DMRs
    a_hypo = rate_delta[rate_delta < -dmr_rate_delta].sort_values()[:100000]
    b_hypo = rate_delta[rate_delta > dmr_rate_delta].sort_values(ascending=False)[:100000]
    
    sparse_data = csr_matrix(dmr_rate.index.isin(a_hypo.index | b_hypo.index)[None, :])

    pairs.append([a, b])
    records.append(sparse_data)
    print(i, a, a_hypo.size, f'{a_hypo.iat[-1]:.2f}', b, b_hypo.size, f'{b_hypo.iat[-1]:.2f}', sep='\t')

0	Foxp2_Trpc7	34112	-0.40	Foxp2_Dchs2	4919	0.40
1	Foxp2_Trpc7	50759	-0.40	MSN-D2_Nrp2	38380	0.40
2	Foxp2_Trpc7	32865	-0.40	MSN-D2_Casz1	25913	0.40
3	Foxp2_Trpc7	75855	-0.40	D1L-Fstl4_Cadm1	38939	0.40
4	Foxp2_Trpc7	83935	-0.40	D1L-Fstl4_Crim1	58012	0.40
5	Foxp2_Trpc7	46945	-0.40	MSN-D2_Slc24a2	60125	0.40
6	Foxp2_Trpc7	37574	-0.40	MSN-D2_Col14a1	46083	0.40
7	Foxp2_Trpc7	34833	-0.40	MSN-D1_Khdrbs3	69547	0.40
8	Foxp2_Trpc7	46867	-0.40	MSN-D1_Hrh1	87735	0.40
9	Foxp2_Trpc7	33305	-0.40	D1L-Fstl4_Trps1	47424	0.40
10	Foxp2_Trpc7	41086	-0.40	D1L-Fstl4_Grm3	70052	0.40
11	Foxp2_Trpc7	59001	-0.40	MSN-D1_Plxnc1	62605	0.40
12	Foxp2_Trpc7	57556	-0.40	MSN-D1_Ntn1	60316	0.40
13	Foxp2_Trpc7	100000	-0.66	OLF_Xkr6	100000	0.58
14	Foxp2_Trpc7	100000	-0.63	OLF_Mapk10	100000	0.43
15	Foxp2_Trpc7	100000	-0.64	OLF_Pag1	100000	0.48
16	Foxp2_Trpc7	100000	-0.60	OLF_Gabbr2	93285	0.40
17	Foxp2_Trpc7	100000	-0.52	OLF_Trpc4	83852	0.40
18	Foxp2_Trpc7	100000	-0.55	OLF_Kcnd3	71224	0.40
19	Foxp2_Trpc7	100000	-0.51	Foxp2_Hom

154	MSN-D2_Nrp2	8793	-0.40	MSN-D2_Slc24a2	18862	0.40
155	MSN-D2_Nrp2	15483	-0.40	MSN-D2_Col14a1	21408	0.40
156	MSN-D2_Nrp2	22795	-0.40	MSN-D1_Khdrbs3	63296	0.40
157	MSN-D2_Nrp2	18451	-0.40	MSN-D1_Hrh1	59681	0.40
158	MSN-D2_Nrp2	8945	-0.40	D1L-Fstl4_Trps1	25135	0.40
159	MSN-D2_Nrp2	9154	-0.40	D1L-Fstl4_Grm3	27420	0.40
160	MSN-D2_Nrp2	8264	-0.40	MSN-D1_Plxnc1	15775	0.40
161	MSN-D2_Nrp2	20116	-0.40	MSN-D1_Ntn1	30824	0.40
162	MSN-D2_Nrp2	100000	-0.62	OLF_Xkr6	100000	0.61
163	MSN-D2_Nrp2	100000	-0.59	OLF_Mapk10	100000	0.45
164	MSN-D2_Nrp2	100000	-0.60	OLF_Pag1	100000	0.50
165	MSN-D2_Nrp2	100000	-0.56	OLF_Gabbr2	100000	0.41
166	MSN-D2_Nrp2	100000	-0.49	OLF_Trpc4	100000	0.40
167	MSN-D2_Nrp2	100000	-0.52	OLF_Kcnd3	83413	0.40
168	MSN-D2_Nrp2	100000	-0.48	Foxp2_Homer2	50568	0.40
169	MSN-D2_Nrp2	100000	-0.44	Foxp2_Inpp4b	39458	0.40
170	MSN-D2_Nrp2	68486	-0.40	D1L-Fstl4_Sipa1l2	47983	0.40
171	MSN-D2_Nrp2	100000	-0.49	D1L-PAL_Flrt2	56316	0.40
172	MSN-D2_Nrp2	100000	-0.48	D1L-PAL_Plcxd3	47339	0.40
1

303	D1L-Fstl4_Cadm1	18330	-0.40	D1L-Fstl4_Trps1	63913	0.40
304	D1L-Fstl4_Cadm1	17411	-0.40	D1L-Fstl4_Grm3	61825	0.40
305	D1L-Fstl4_Cadm1	25833	-0.40	MSN-D1_Plxnc1	53561	0.40
306	D1L-Fstl4_Cadm1	34059	-0.40	MSN-D1_Ntn1	63487	0.40
307	D1L-Fstl4_Cadm1	100000	-0.50	OLF_Xkr6	100000	0.60
308	D1L-Fstl4_Cadm1	100000	-0.48	OLF_Mapk10	100000	0.45
309	D1L-Fstl4_Cadm1	100000	-0.49	OLF_Pag1	100000	0.50
310	D1L-Fstl4_Cadm1	100000	-0.43	OLF_Gabbr2	96661	0.40
311	D1L-Fstl4_Cadm1	100000	-0.41	OLF_Trpc4	100000	0.42
312	D1L-Fstl4_Cadm1	100000	-0.42	OLF_Kcnd3	81515	0.40
313	D1L-Fstl4_Cadm1	87020	-0.40	Foxp2_Homer2	46951	0.40
314	D1L-Fstl4_Cadm1	76928	-0.40	Foxp2_Inpp4b	38317	0.40
315	D1L-Fstl4_Cadm1	48912	-0.40	D1L-Fstl4_Sipa1l2	49834	0.40
316	D1L-Fstl4_Cadm1	72229	-0.40	D1L-PAL_Flrt2	36401	0.40
317	D1L-Fstl4_Cadm1	59553	-0.40	D1L-PAL_Plcxd3	23188	0.40
318	D1L-Fstl4_Cadm1	100000	-0.50	CGE-Vip_Ntng1	87323	0.40
319	D1L-Fstl4_Cadm1	100000	-0.48	Unc5c_Unc5c	100000	0.65
320	D1L-Fstl4_Cadm1	100000	-0.43	CGE-Lam

444	MSN-D2_Slc24a2	27527	-0.40	D1L-Fstl4_Trps1	34254	0.40
445	MSN-D2_Slc24a2	9754	-0.40	D1L-Fstl4_Grm3	20000	0.40
446	MSN-D2_Slc24a2	5543	-0.40	MSN-D1_Plxnc1	4990	0.40
447	MSN-D2_Slc24a2	21079	-0.40	MSN-D1_Ntn1	18027	0.40
448	MSN-D2_Slc24a2	100000	-0.68	OLF_Xkr6	100000	0.60
449	MSN-D2_Slc24a2	100000	-0.66	OLF_Mapk10	100000	0.45
450	MSN-D2_Slc24a2	100000	-0.67	OLF_Pag1	100000	0.50
451	MSN-D2_Slc24a2	100000	-0.65	OLF_Gabbr2	100000	0.42
452	MSN-D2_Slc24a2	100000	-0.58	OLF_Trpc4	100000	0.42
453	MSN-D2_Slc24a2	100000	-0.62	OLF_Kcnd3	88358	0.40
454	MSN-D2_Slc24a2	100000	-0.59	Foxp2_Homer2	61252	0.40
455	MSN-D2_Slc24a2	100000	-0.56	Foxp2_Inpp4b	49612	0.40
456	MSN-D2_Slc24a2	100000	-0.42	D1L-Fstl4_Sipa1l2	58026	0.40
457	MSN-D2_Slc24a2	100000	-0.59	D1L-PAL_Flrt2	62047	0.40
458	MSN-D2_Slc24a2	100000	-0.59	D1L-PAL_Plcxd3	55210	0.40
459	MSN-D2_Slc24a2	100000	-0.68	CGE-Vip_Ntng1	93396	0.40
460	MSN-D2_Slc24a2	100000	-0.66	Unc5c_Unc5c	100000	0.66
461	MSN-D2_Slc24a2	100000	-0.62	CGE-Lamp5_Grid1	100000

587	MSN-D1_Khdrbs3	100000	-0.76	OLF_Pag1	100000	0.52
588	MSN-D1_Khdrbs3	100000	-0.74	OLF_Gabbr2	100000	0.44
589	MSN-D1_Khdrbs3	100000	-0.66	OLF_Trpc4	100000	0.44
590	MSN-D1_Khdrbs3	100000	-0.71	OLF_Kcnd3	91346	0.40
591	MSN-D1_Khdrbs3	100000	-0.68	Foxp2_Homer2	63233	0.40
592	MSN-D1_Khdrbs3	100000	-0.65	Foxp2_Inpp4b	49875	0.40
593	MSN-D1_Khdrbs3	100000	-0.49	D1L-Fstl4_Sipa1l2	48142	0.40
594	MSN-D1_Khdrbs3	100000	-0.71	D1L-PAL_Flrt2	74659	0.40
595	MSN-D1_Khdrbs3	100000	-0.70	D1L-PAL_Plcxd3	67642	0.40
596	MSN-D1_Khdrbs3	100000	-0.78	CGE-Vip_Ntng1	97319	0.40
597	MSN-D1_Khdrbs3	100000	-0.75	Unc5c_Unc5c	100000	0.69
598	MSN-D1_Khdrbs3	100000	-0.71	CGE-Lamp5_Grid1	100000	0.76
599	MSN-D1_Khdrbs3	100000	-0.70	CGE-Lamp5_Dock5	100000	0.79
600	MSN-D1_Khdrbs3	100000	-0.70	CGE-Lamp5_Grk5	100000	0.78
601	MSN-D1_Khdrbs3	100000	-0.76	CGE-Lamp5_Sorcs1	100000	0.68
602	MSN-D1_Khdrbs3	100000	-0.77	CGE-Lamp5_Nrxn3	100000	0.61
603	MSN-D1_Khdrbs3	100000	-0.79	CGE-Vip_Ptprm	95343	0.40
604	MSN-D1_Khdrbs3	100000	-

...

2331	MGE-Pvalb_Ptprk	27289	-0.40	MGE-Pvalb_Gfra2	39438	0.40
2332	MGE-Pvalb_Ptprk	23695	-0.40	MGE-Pvalb_Cnih3	28389	0.40
2333	MGE-Pvalb_Ptprk	43216	-0.40	MGE-Pvalb_Entpd3	48964	0.40
2334	MGE-Pvalb_Ptprk	40928	-0.40	MGE-Pvalb_Cacna1i	100000	0.46
2335	MGE-Pvalb_Ptprk	40128	-0.40	MGE-Pvalb_Thsd7a	80267	0.40
2336	MGE-Pvalb_Ptprk	35759	-0.40	MGE-Pvalb_Sema5a	68285	0.40
2337	MGE-Pvalb_Ptprk	46096	-0.40	MGE-Sst_Ptpre	66838	0.40
2338	MGE-Pvalb_Ptprk	59553	-0.40	MGE-Sst_Unc5b	92582	0.40
2339	MGE-Pvalb_Ptprk	68354	-0.40	MGE-Sst_Rxra	41796	0.40
2340	MGE-Pvalb_Ptprk	79706	-0.40	MGE-Sst_Bmper	49481	0.40
2341	MGE-Pvalb_Ptprk	56702	-0.40	MGE-Sst_Kcnip4	59724	0.40
2342	MGE-Pvalb_Ptprk	47771	-0.40	MGE-Sst_Ubtd1	38907	0.40
2343	MGE-Pvalb_Ptprk	65909	-0.40	MGE-Sst_Etv1	57160	0.40
2344	MGE-Pvalb_Ptprk	60351	-0.40	MGE-Sst_Dock4	82876	0.40
2345	MGE-Pvalb_Ptprk	70474	-0.40	MGE-Sst_Frmd6	52202	0.40
2346	MGE-Pvalb_Ptprk	66078	-0.40	MGE-Sst_Rerg	80960	0.40
2347	MGE-Pvalb_Ptprk	83624	-0.40	PAL-Inh_Deptor	100000	0

2469	MGE-Pvalb_Cacna1i	100000	-0.53	MGE-Sst_Etv1	48954	0.40
2470	MGE-Pvalb_Cacna1i	100000	-0.53	MGE-Sst_Dock4	76909	0.40
2471	MGE-Pvalb_Cacna1i	100000	-0.49	MGE-Sst_Frmd6	32309	0.40
2472	MGE-Pvalb_Cacna1i	100000	-0.48	MGE-Sst_Rerg	57128	0.40
2473	MGE-Pvalb_Cacna1i	100000	-0.60	PAL-Inh_Deptor	97139	0.40
2474	MGE-Pvalb_Cacna1i	100000	-0.63	LSX-Inh_Lats2	100000	0.66
2475	MGE-Pvalb_Cacna1i	100000	-0.69	LSX-Inh_Nxph1	100000	0.42
2476	MGE-Pvalb_Cacna1i	100000	-0.69	LSX-Inh_Foxp2	100000	0.50
2477	MGE-Pvalb_Cacna1i	100000	-0.67	LSX-Inh_Enox1	100000	0.53
2478	MGE-Pvalb_Cacna1i	100000	-0.65	LSX-Inh_Zeb2	100000	0.59
2479	MGE-Pvalb_Cacna1i	100000	-0.62	LSX-Inh_Cacna1i	57505	0.40
2480	MGE-Pvalb_Cacna1i	100000	-0.69	LSX-Inh_Dock10	66212	0.40
2481	MGE-Pvalb_Cacna1i	100000	-0.70	PAL-Inh_Chat	61230	0.40
2482	MGE-Pvalb_Cacna1i	100000	-0.65	PAL-Inh_Meis2	3128	0.40
2483	MGE-Pvalb_Cacna1i	100000	-0.70	PAL-Inh_Tcf7l2	37144	0.40
2484	MGE-Pvalb_Cacna1i	100000	-0.70	PAL-Inh_Onecut2	43655	0.40
2485	MGE-Pvalb_Ca

2608	MGE-Sst_Rxra	66326	-0.40	PAL-Inh_Deptor	100000	0.45
2609	MGE-Sst_Rxra	59051	-0.40	LSX-Inh_Lats2	100000	0.67
2610	MGE-Sst_Rxra	77653	-0.40	LSX-Inh_Nxph1	100000	0.45
2611	MGE-Sst_Rxra	76058	-0.40	LSX-Inh_Foxp2	100000	0.52
2612	MGE-Sst_Rxra	70563	-0.40	LSX-Inh_Enox1	100000	0.54
2613	MGE-Sst_Rxra	63938	-0.40	LSX-Inh_Zeb2	100000	0.59
2614	MGE-Sst_Rxra	69363	-0.40	LSX-Inh_Cacna1i	62060	0.40
2615	MGE-Sst_Rxra	76174	-0.40	LSX-Inh_Dock10	68071	0.40
2616	MGE-Sst_Rxra	73703	-0.40	PAL-Inh_Chat	66703	0.40
2617	MGE-Sst_Rxra	77330	-0.40	PAL-Inh_Meis2	2766	0.40
2618	MGE-Sst_Rxra	78738	-0.40	PAL-Inh_Tcf7l2	38723	0.40
2619	MGE-Sst_Rxra	79757	-0.40	PAL-Inh_Onecut2	49484	0.40
2620	MGE-Sst_Rxra	68778	-0.40	PAL-Inh_Meis1	100000	0.44
2621	MGE-Sst_Rxra	71476	-0.40	PAL-Inh_Tmem178	29715	0.40
2622	MGE-Sst_Rxra	72998	-0.40	PAL-Inh_Igdcc3	15464	0.40
2623	MGE-Sst_Rxra	76639	-0.40	PAL-Inh_Ptprd	25000	0.40
2624	MGE-Sst_Rxra	51424	-0.40	MGE-Sst_Chodl	84181	0.40
2625	MGE-Sst_Rxra	67790	-0.40	PAL-Inh_Rarb	45793	0.

2753	MGE-Sst_Frmd6	53189	-0.40	MGE-Sst_Chodl	75874	0.40
2754	MGE-Sst_Frmd6	80830	-0.40	PAL-Inh_Rarb	41655	0.40
2755	MGE-Sst_Rerg	100000	-0.46	PAL-Inh_Deptor	100000	0.45
2756	MGE-Sst_Rerg	100000	-0.48	LSX-Inh_Lats2	100000	0.70
2757	MGE-Sst_Rerg	100000	-0.56	LSX-Inh_Nxph1	100000	0.46
2758	MGE-Sst_Rerg	100000	-0.57	LSX-Inh_Foxp2	100000	0.55
2759	MGE-Sst_Rerg	100000	-0.52	LSX-Inh_Enox1	100000	0.55
2760	MGE-Sst_Rerg	100000	-0.50	LSX-Inh_Zeb2	100000	0.62
2761	MGE-Sst_Rerg	100000	-0.52	LSX-Inh_Cacna1i	67646	0.40
2762	MGE-Sst_Rerg	100000	-0.56	LSX-Inh_Dock10	72577	0.40
2763	MGE-Sst_Rerg	100000	-0.57	PAL-Inh_Chat	72529	0.40
2764	MGE-Sst_Rerg	100000	-0.54	PAL-Inh_Meis2	3257	0.40
2765	MGE-Sst_Rerg	100000	-0.58	PAL-Inh_Tcf7l2	41996	0.40
2766	MGE-Sst_Rerg	100000	-0.59	PAL-Inh_Onecut2	52465	0.40
2767	MGE-Sst_Rerg	100000	-0.45	PAL-Inh_Meis1	100000	0.44
2768	MGE-Sst_Rerg	100000	-0.47	PAL-Inh_Tmem178	24917	0.40
2769	MGE-Sst_Rerg	100000	-0.54	PAL-Inh_Igdcc3	17989	0.40
2770	MGE-Sst_Rerg	100000	-0.50	PAL-

2897	PAL-Inh_Meis2	1398	-0.40	PAL-Inh_Rarb	52147	0.40
2898	PAL-Inh_Tcf7l2	36092	-0.40	PAL-Inh_Onecut2	43949	0.40
2899	PAL-Inh_Tcf7l2	26833	-0.40	PAL-Inh_Meis1	100000	0.41
2900	PAL-Inh_Tcf7l2	39782	-0.40	PAL-Inh_Tmem178	35800	0.40
2901	PAL-Inh_Tcf7l2	35480	-0.40	PAL-Inh_Igdcc3	15481	0.40
2902	PAL-Inh_Tcf7l2	40830	-0.40	PAL-Inh_Ptprd	29697	0.40
2903	PAL-Inh_Tcf7l2	37192	-0.40	MGE-Sst_Chodl	100000	0.44
2904	PAL-Inh_Tcf7l2	36352	-0.40	PAL-Inh_Rarb	53984	0.40
2905	PAL-Inh_Onecut2	48147	-0.40	PAL-Inh_Meis1	100000	0.48
2906	PAL-Inh_Onecut2	51602	-0.40	PAL-Inh_Tmem178	38172	0.40
2907	PAL-Inh_Onecut2	47529	-0.40	PAL-Inh_Igdcc3	19185	0.40
2908	PAL-Inh_Onecut2	55233	-0.40	PAL-Inh_Ptprd	33903	0.40
2909	PAL-Inh_Onecut2	49346	-0.40	MGE-Sst_Chodl	100000	0.45
2910	PAL-Inh_Onecut2	51757	-0.40	PAL-Inh_Rarb	60225	0.40
2911	PAL-Inh_Meis1	100000	-0.41	PAL-Inh_Tmem178	20119	0.40
2912	PAL-Inh_Meis1	100000	-0.42	PAL-Inh_Igdcc3	10208	0.40
2913	PAL-Inh_Meis1	100000	-0.43	PAL-Inh_Ptprd	17490	0.40
2914	PAL-Inh_Me

In [10]:
adata = anndata.AnnData(X=vstack(records),
                        obs=pd.DataFrame(pairs, columns=['cluster_a', 'cluster_b']),
                        var=pd.DataFrame([], index=dmr_rate.index))

Transforming to str index.


In [11]:
adata.write_h5ad('PairwiseDMR.h5ad')

... storing 'cluster_a' as categorical
... storing 'cluster_b' as categorical
